In [ ]:
import os, sys
import time
import matplotlib
import matplotlib.pyplot as plt
import random

parentPath = os.path.abspath("../..")
if parentPath not in sys.path:
    sys.path.insert(0, parentPath)

import model as modellib
import visualize
from model import log

%matplotlib inline 

# Root directory of the project
ROOT_DIR = parentPath

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
WEIGHTS_DIR = os.path.join(MODEL_DIR, "nyu_depth_v2_scenenet20171121T1912/mask_rcnn_nyu_depth_v2_scenenet_0605.h5")

CUSTOM_DIR = "/home/orestisz/repositories/Mask_RCNN/instance_segmentation/custom_data/static_images"

# import SceneNet
from dataset import *

def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
# Validation dataset
dataset = Dataset()
dataset.load(CUSTOM_DIR)
dataset.prepare()

## Detection

In [ ]:
class InferenceConfig(Config):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", config=inference_config, model_dir=MODEL_DIR)
model.load_weights(WEIGHTS_DIR, by_name=True)

In [ ]:
# Test on a random image
image_id = random.choice(dataset.image_ids)
# image_id = 11
# image_id = 56
original_image = dataset.load_image(image_id, mode="RGBD")

print(dataset.image_info[image_id]['path'])
image = original_image[:, :, 0:3]
depth = original_image[:, :, 3]
depth_raw = original_image[:, :, 4]
mask = original_image[:, :, 5]

print("image_id", image_id)
log("image", image)
log("depth", depth)
log("depth_raw", depth_raw)
log("mask", mask)

depth_norm = depth / np.max(depth)
rgbd = np.dstack((
    image[:, :, 0] * depth_norm,
    image[:, :, 1] * depth_norm,
    image[:, :, 2] * depth_norm))

ax = get_ax()
ax.axis('off')
ax.imshow(np.dstack(depth_raw / np.max(depth_raw))

ax = get_ax()
ax.axis('off')
ax.imshow(depth_norm)

ax = get_ax()
ax.axis('off')
ax.imshow(rgbd.astype(np.uint8))

ax = get_ax()
ax.axis('off')
ax.imshow(image.astype(np.uint8))

In [ ]:
import cv2

for i in range(19):
    image_id = dataset.image_ids[i]
    original_image = dataset.load_image(image_id, mode="RGBD")
    
    #  flip some of the images since they are on the head
    if i > 15:
        original_image = cv2.flip(original_image, 0)
    
    image = original_image[:, :, 0:3]
    depth = original_image[:, :, 3]
    depth_raw = original_image[:, :, 4]
    mask = original_image[:, :, 5]

    ax = get_ax()
    ax.axis('off')
    ax.imshow(np.dstack((255 - depth_raw,) * 3).astype(np.uint8))
    
    ax = get_ax()
    ax.axis('off')
    ax.imshow(np.dstack((255 - depth,) * 3).astype(np.uint8))
    
    ax = get_ax()
    ax.axis('off')
    ax.imshow(image.astype(np.uint8))
    
    #  comment out to test on grey image 
#     original_image = np.dstack((np.ones(image.shape) * 255 / 2.0, depth))
    #  comment out to test on grey depth 
#     original_image = np.dstack((image, np.ones(depth.shape) * 255 / 2.0))

    start = time.clock()
    r = model.detect([original_image[:, :, 0:4]])[0]
    print(time.clock() - start)

    #  comment out to show individual masks  
#     for i, roi in enumerate(r['rois']):
#         roi = np.array([roi])
#         mask = np.expand_dims(np.array(r['masks'][:, :, i]), 2)
#         class_id = np.array([r['class_ids'][i]])
#         score = np.array([r['scores'][i]])
#         visualize.display_instances(image, roi, mask, class_id, 
#                         dataset.class_names, score, ax=get_ax(), colors=[[0, 1, 0]])

    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                        dataset.class_names, r['scores'], ax=get_ax())